In [1]:
import sys; sys.path.append("../")
import pandas as pd

from lib.explore import show_unique
from lib.split_schema import detect_unique_columns_subset
from clean.ipm_new_orleans_pd_cprr_allegations import clean
from clean.ipm_new_orleans_pd_cprr_actions_taken import clean as clean_at


In [2]:
officers, citizens, complaints, allegations, allegation_citizens = clean()
at_allegations, action_takens = clean_at()

## Questions
- What is `pib_control_number`? Is it the same as `tracking_number`?
- Does `officer_primary_key` reference `officer_number` in PPRR?
- What is `officer_unknown_id`?
- There are 2474 rows with empty `officer_primary_key`, should we keep them?
- Should we keep `citizens` as a table? What should we name it?
- There are 2049 rows with empty `allegation_primary_key`, and none of them are sustained. Should we keep them?
- Is `status` column `investigation_status`?
- Go through each field in allegations and decide whether we want to keep them

In [3]:
allegations

,pib_control_number,officer_primary_key,allegation_primary_key,allegation,allegation_finding,allegation_final_disposition,all_findings,disposition_oipm_by_officer,allegation_finding_oipm,allegation_1,...,officer_title,officer_age_at_time_of_uof,officer_years_exp_at_time_of_uof,officer_years_with_unit,officer_type,officer_employment_status,officer_department,officer_division,officer_sub_division_a,officer_sub_division_b
559,2011-0001-N,2326,1,paragraph 04 - neglect of duty,nfim case,nfim case,nfim case,nfim,nfim,neglect of duty,...,POLICE SERGEANT,45.0,18.0,-3.0,Commissioned,Active,FOB - Field Operations Bureau,First District,2nd Platoon,NaN
560,2011-0003-C,3667,2,paragraph 01 - professionalism,nfim case,nfim case,nfim case,nfim,nfim,professionalism,...,POLICE SERGEANT,45.0,24.0,-3.0,Commissioned,Active,FOB - Field Operations Bureau,Special Operations Division,Traffic Section,DWI Unit
561,2011-0015-R,1404,4,paragraph 02 - instructions from authoritative...,sustained,sustained,sustained,sustained,sustained,instructions from authoritative source,...,POLICE OFFICER,29.0,3.0,-1.0,Commissioned,Terminated,MSB - Management Service Bureau,Administrative Duties Services,Staff,NaN
562,2011-0015-R,974,3,paragraph 02 - instructions from authoritative...,sustained,sustained,sustained,sustained,sustained,instructions from authoritative source,...,SENIOR POLICE OFFICER,23.0,2.0,-1.0,Commissioned,Active,ISB - Investigations and Support Bureau,Special Investigations Division,Street Gang Unit,Squad A
563,2011-0335-R,1220,5,paragraph 04 - neglect of duty,exonerated,exonerated,exonerated,exonerated,exonerated,neglect of duty,...,POLICE OFFICER 3,27.0,3.0,-2.0,Commissioned,Terminated,FOB - Field Operations Bureau,Seventh District,Staff,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26320,2019-0080-P,1025,26288,paragraph 04 - neglect of duty,pending investigation,,pending investigation,pending,pending,neglect of duty,...,SENIOR POLICE OFFICER,40.0,11.0,7.0,Commissioned,Active,FOB - Field Operations Bureau,Second District,B Platoon,NaN
26321,2019-0112-P,882,26291,paragraph 04 - neglect of duty,,,,pending,pending,neglect of duty,...,SENIOR POLICE OFFICER,45.0,10.0,5.0,Commissioned,Active,FOB - Field Operations Bureau,Sixth District,C Platoon,NaN
26322,2019-0112-P,5404,26292,paragraph 04 - neglect of duty,,,,pending,pending,neglect of duty,...,SENIOR POLICE OFFICER,36.0,5.0,4.0,Commissioned,Active,FOB - Field Operations Bureau,Sixth District,D.I.U.,Persons
26323,2019-0112-P,3719,26293,paragraph 04 - neglect of duty,,,,pending,pending,neglect of duty,...,POLICE SERGEANT,52.0,31.0,4.0,Commissioned,Active,FOB - Field Operations Bureau,Sixth District,Task Force,B


In [6]:
print(allegations.columns)
allegations.loc[allegations.allegation_primary_key == "1", [
    "allegation", "allegation_finding", "allegation_final_disposition", "allegation_created_on", "allegation_finding_date",
    "allegation_final_disposition_date", "allegation_directive"
]]

Index(['pib_control_number', 'officer_primary_key', 'allegation_primary_key',
       'allegation', 'allegation_finding', 'allegation_final_disposition',
       'all_findings', 'disposition_oipm_by_officer',
       'allegation_finding_oipm', 'allegation_1', 'allegation_class',
       'allegation_finding_date', 'allegation_created_on',
       'allegation_class_1', 'allegation_final_disposition_date',
       'allegation_directive', 'allegation_alert_processed',
       'allegation_alert_processed_date', 'citizen_age', 'citizen_num_shots',
       'is_anonymous', 'officer_badge_number', 'officer_current_supervisor',
       'officer_title', 'officer_age_at_time_of_uof',
       'officer_years_exp_at_time_of_uof', 'officer_years_with_unit',
       'officer_type', 'officer_employment_status', 'officer_department',
       'officer_division', 'officer_sub_division_a', 'officer_sub_division_b'],
      dtype='object')


,allegation,allegation_finding,allegation_final_disposition,allegation_created_on,allegation_finding_date,allegation_final_disposition_date,allegation_directive
559,paragraph 04 - neglect of duty,nfim case,nfim case,3/25/11 0:00,1/29/11 0:00,5/10/11,rule 4: performance of duty - paragraph 04 - n...


In [11]:
at_allegations.loc[:, 'allegation_primary_key'] = at_allegations.allegation_primary_key.fillna(0).astype("int64").astype(str).str.replace(r"^0$", "", regex=True)
at_allegations.loc[~at_allegations.allegation_primary_key.isin(allegations.allegation_primary_key.to_list()), "allegation_year_created"].unique()

array([  nan, 2016., 2018., 2019.])

In [12]:
allegations.allegation_created_on.unique()

array(['3/25/11 0:00', '3/28/11 0:00', '4/6/11 0:00', ..., '2/20/19 0:00',
       '2/15/19 0:00', '2/21/19 0:00'], dtype=object)